### Import Feature Extractor
- input shape: 299x299x3
- feature shape: 8x8x2048

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
import tensorflow as tf

In [2]:
pretrained_model = InceptionV3(include_top=False,
                               weights='imagenet')

In [3]:
feature_extractor = tf.keras.Model(pretrained_model.input,
                                   pretrained_model.layers[-1].output)

### JPEG Files

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
jpeg_files = os.listdir('train2014')
jpeg_files = [os.path.join('train2014', f) for f in jpeg_files]

In [6]:
len(jpeg_files)

82783

### Save Features

In [7]:
from tqdm import tqdm

In [8]:
save_main = 'train_inceptionv3_feature'

In [9]:
# for file in tqdm(jpeg_files):
#     # load image
#     img = plt.imread(file)
#     if len(img.shape) != 3:
#         img = np.concatenate([img[...,np.newaxis]]*3, axis=-1)
#     img = cv2.resize(img, (299, 299))
#     # get feature map (8,8,2048)
#     img = preprocess_input(img)
#     fm = feature_extractor(img[np.newaxis, ...])[0]
#     # get jpeg file name
#     name = file.split('\\')[-1].split('.')[0]
#     save_to = os.path.join(save_main, name)
#     np.save(save_to, fm)

In [10]:
def load_image(jpeg_path):
    img = tf.io.read_file(jpeg_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = preprocess_input(img)
    return img, jpeg_path

In [11]:
jpegfile_dataset = tf.data.Dataset.from_tensor_slices(jpeg_files)
jpegfile_dataset = jpegfile_dataset.map(load_image,
                   num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(32)
for img, path in tqdm(jpegfile_dataset):
    batch_features = feature_extractor(img)
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1, batch_features.shape[3]))
    for bf, p in zip(batch_features, path):
        name = p.numpy().decode('utf-8').split('\\')[-1].split('.')[0]
        save_to = os.path.join(save_main, name)
        np.save(save_to, bf.numpy())

2587it [1:43:56,  2.41s/it]
